In [ ]:
#import all tools 

#EDA tools plotting tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline #for plots to appear inside the notebook

#Machine learning tools

#Modules from Scikit-learn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

#Model Evaluation
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCv, GridSearchCv
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, fi_score
from sklearn.metrics import plot_roc_curve